In [1]:
import numpy as np
import pandas as pd 
def precisionMatrix(proba, y):
    def _precisionClassify(df,proba, wins, c =0 ):
        for indx, v in enumerate(proba):
            row = 0
            col = 0
            if wins[indx] == c:
                col = 0
            else:
                col =1
            if v <0.2:
                row =6 
            elif v < 0.3 and  v >=0.2:
                row =5 
            elif v < 0.4 and v >= 0.3:
                row = 4 
            elif v < 0.5 and v >= 0.4:
                row = 3 
            elif v < 0.6 and v >= 0.5:
                row = 2 
            elif v < 0.8 and v >= 0.6:
                row = 1
            df.iloc[row,col] = df.iloc[row,col]+1 
        df[df.columns[2]] = df[df.columns[0]] /(df[df.columns[1]] + df[df.columns[0]])
    return df
    rowHeader = ['>80','60-80','50-60','40-50','30-40','20-30','<20']
    df = pd.DataFrame(np.zeros(shape=(7,3)),index=rowHeader, columns=['h_Correct', 'h_Wrong','h_Precent'])
    hproba = proba[:,0]
    wins = np.argmax(y,axis=1)
    df = _precisionClassify(df,hproba,wins)
    temp = pd.DataFrame(np.zeros(shape=(7,3)),index=rowHeader, columns=['d_Correct', 'd_Wrong','d_Precent'])
    dproba = proba[:,1]
    df = df.join(_precisionClassify(temp,dproba,wins,c=1))
    temp = pd.DataFrame(np.zeros(shape=(7,3)),index=rowHeader, columns=['a_Correct', 'a_Wrong','a_Precent'])
    aproba = proba[:,2]
    df = df.join(_precisionClassify(temp,aproba,wins,c=2))
    
    bound = pd.DataFrame(np.array([[0.8,1.0],[0.6,0.8],[0.5,0.6],[0.4,0.5],[0.3,0.4],[0.2,0.3],[0,0.2]] )
                                ,index=rowHeader, columns=['[lower', 'upper)'])
            
    return bound.join(df)
       

In [2]:
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
class FootballDataHelper:
    def __init__ (self, recentNum=5):
        self.win_mapping = {'H':0, 'D':1,'A':2}
        self.recentNum = recentNum
        self.df = None
        self.teamsData={}
        self.session = 0
        #self.hiddensCount = 2
       
        
    def readFootBallData(self,filename): 
        df = pd.read_csv(filename)
        #df = df.drop(df.columns[range(23,df.shape[1])], axis=1)
        #df = df.drop("Div",axis=1)
        df['Date'] = pd.to_datetime(df['Date'],dayfirst=True)
        df['session'] = pd.Series(np.ones(shape=(df.shape[0],))*self.session, index=df.index)
        self.session = self.session +1
        #df= df.drop('Referee', 1)
        #self.team = df['HomeTeam'].drop_duplicates()
        if self.df is None:
            self.df = df
        else:
            self.df = pd.concat([self.df,df])
            
        self.df = self.df.reset_index(drop=True)
        teams = self.df['HomeTeam'].drop_duplicates()
        teamMap = {}
        for index , v in enumerate(teams):
            teamMap[v] = index
        self.teamsMap = teamMap
        referees = self.df['Referee'].drop_duplicates()
        refereesMap = {}
        for index , v in enumerate(referees):
            refereesMap[v] = index+1

        self.refereesMap = refereesMap
 
    def getTeam(self,dataFrame, teamName):       
        return dataFrame[(dataFrame["HomeTeam"] == teamName) | (dataFrame["AwayTeam"] == teamName)]
 
        
    def previousRecords(self,team, date , recentNum):
        prev = team[( team["Date"] < date)]
        
        if prev.shape[0] < recentNum :
            #print("less than min Num")
            return None
        else:
            return prev.iloc[-recentNum:]
    def readPredict (self, filename):
        df = pd.read_csv(filename)
        df['Date'] = pd.to_datetime(df['Date'],dayfirst=True)

        return df
    
    
  
    def inverseTeamMapping (self, col):
        inverseMap ={}
        for name in self.teamsMap.keys():        
            inverseMap[self.teamsMap[name]] = name
        res =[]
        for idex, v in enumerate(col):
            res.append(inverseMap[v])
        return res


    def readTeamMatch(self, teamName):
        df = pd.read_csv('teams/'+teamName+'.csv')
        df['1'] = pd.to_datetime(df['1'],yearfirst=True)
        #df['1']= (pd.to_numeric(df['1'])/1e9/24/60/60)
        self.teamsData[teamName]=df.sort(['1'],ascending=[False])
        self.teamsById[self.teamsMap[teamName]]=self.teamsData[teamName]
    
    def commonMapping(self, X):
        X['HomeTeam'] = X['HomeTeam'].map(self.teamsMap)
        X['AwayTeam'] = X['AwayTeam'].map(self.teamsMap)
        X['Referee']=X['Referee'].map(self.refereesMap).fillna(0)
        X['HTR'] = X['HTR'].map(self.win_mapping)
        X['FTR'] = X['FTR'].map(self.win_mapping)
        return X
    def initData(self, X, target,encode):
        X  = X.sort_values(by="Date")
        isInput = False
        if target is None:
            target =X      
        else:
            if self.ohe is None:
                raise Exception("Not yet get train data")
            isInput = True
            if encode == True:
                target = self.commonMapping(target)
        y=None
        if encode == True:    
            X =self.commonMapping(X)
            y = []
            for v in target['FTR']:
                y.append(range(3)==v)
        else:
            y = target['FTR'].values
        target_date = (pd.to_numeric(target['Date'])/1e9/24/60/60).values
        return isInput, X,y, target, target_date
   
    def aggregate(self,recents,nonExpand,isInput,encode):
        res =None
        if encode == True:
            if isInput==False:
                self.ohe = OneHotEncoder(categorical_features='all')
                self.ohe.fit(recents)
            res = self.ohe.transform(recents).toarray()
        else:
            res = np.array(recents)
        self.dateColumn = res.shape[1]
        res = np.hstack([res,nonExpand])
        return res
  
    def getH7(self,removeInsufficient=False, target=None,encode = True):
        #Simple recent win,draw, lose 
        df = self.df
        if removeInsufficient == True:
            df= df[df['Sufficient'] == 1]
        
        
        isInput, X, y,target, target_date = self.initData(df,target,encode)
        resy=[]
        resx=[]
        print("start format")
        recents = X[['HomeTeam','AwayTeam','Referee']].values
        haccp = X['HAccP'].values.reshape(X.shape[0],1)
        aaccp = X['AAccP'].values.reshape(X.shape[0],1)
        homeRecent = np.hstack([X[['HWin','HDraw','HLose']].values,
                                (X['HScore'].values - X['HConcede'].values).reshape(X.shape[0],1)])
        awayRecent = np.hstack([X[['AWin','ADraw','ALose']].values,
                                (X['AScore'].values - X['AConcede'].values).reshape(X.shape[0],1)])
        homeMoral = X['HMoral'].values.reshape(X.shape[0],1)
        awayMoral = X['AMoral'].values.reshape(X.shape[0],1)
        target_date = target_date.reshape(X.shape[0],1)
        nonExpand =np.hstack([target_date,X[['HRestDay','ARestDay','HAccS','AAccS','HAccST','AAccST']].values,haccp-aaccp,(haccp+1)/(aaccp+1),
                                homeRecent,awayRecent, homeMoral - awayMoral + haccp - aaccp])
        res = self.aggregate(recents,nonExpand,isInput,encode)
        print("finish")
        sys.stdout.flush()
        return res, np.array(y)
    def _getRank(self,x, X,teamName,recentNum):
        team = self.getTeam(X,teamName)
        prev = team[team['Date'] < x['Date']].values      
        for i in range(recentNum):
            pass
    def initRanking(self, n = 20):
        defaultPt = 1
        df = self.df.sort(columns=["Date"],ascending=[False])
        df["HPoints"] = pd.Series(np.zeros(shape=(df.shape[0],)))
        df["APoints"] = pd.Series(np.zeros(shape=(df.shape[0],)))
        df["HAccP"] = pd.Series(np.zeros(shape=(df.shape[0],)))
        df["AAccP"] = pd.Series(np.zeros(shape=(df.shape[0],)))
        hpoints= df["HPoints"].values
        apoints=df["APoints"].values
        ftr = df["FTR"].values
        for i in range(df.shape[0]):
            sys.stdout.write("\r progress {}".format(i))
            sys.stdout.flush()
            if ftr[i] == 'H':
                hpoints[i] = 3
                apoints[i] = 0
            elif ftr[i] == 'D':
                hpoints[i] = 1
                apoints[i] = 1
            else :
                hpoints[i] = 0
                apoints[i] = 3
        df["HPoints"]=hpoints
        df["APoints"]=apoints
        for teamName in self.teamsMap.keys():
            print(teamName)
            team  = df[(df['HomeTeam']==teamName) | (df['AwayTeam'] == teamName)] 
            hometeam = team['HomeTeam'].values
            hpoints = team['HPoints'].values
            apoints = team['APoints'].values
            psum = 0
            haccp = team['HAccP'].values
            aaccp = team['AAccP'].values
        
            for  i in range(0,n):
                if i < hpoints.shape[0]:
                    psum = psum + (hpoints[i] if hometeam[i] == teamName else apoints[i] ) 
                else:
                    psum = psum + defaultPt        
                    
        
            for j in range(team.shape[0]):

                if j+n < hpoints.shape[0]:                     
                    psum = psum + (hpoints[j+n] if hometeam[j+n]==teamName else apoints[j+n])
                else:
                    psum = psum + defaultPt 
                
                psum = psum - (hpoints[j] if hometeam[j]==teamName else apoints[j])
                    
                if hometeam[j] == teamName:
                    haccp[j]=psum
                else:
                    aaccp[j]=psum
            team['HAccp']=haccp
            team['AAccP']=aaccp
            #print(team[['HomeTeam','AwayTeam','HAccP','AAccP']])
            df.update(team)
            
            #print(df[['HomeTeam','AwayTeam','HAccP','AAccP']])
        self.df =df
        return df
    def initRecentData(self, n =5):
        df = self.df.sort(columns=["Date"],ascending=[False])
        df["HWin"] = pd.Series(np.zeros(shape=(df.shape[0],)))
        df["AWin"] = pd.Series(np.zeros(shape=(df.shape[0],)))
        df["HDraw"] = pd.Series(np.zeros(shape=(df.shape[0],)))
        df["ADraw"] = pd.Series(np.zeros(shape=(df.shape[0],)))
        df["HLose"] = pd.Series(np.zeros(shape=(df.shape[0],)))
        df["ALose"] = pd.Series(np.zeros(shape=(df.shape[0],)))
        df["HScore"] = pd.Series(np.zeros(shape=(df.shape[0],)))
        df["AScore"] = pd.Series(np.zeros(shape=(df.shape[0],)))
        df["HConcede"] = pd.Series(np.zeros(shape=(df.shape[0],)))
        df["AConcede"] = pd.Series(np.zeros(shape=(df.shape[0],)))
        df["HMoral"] = pd.Series(np.zeros(shape=(df.shape[0],)))
        df["AMoral"] = pd.Series(np.zeros(shape=(df.shape[0],)))
        df["HRestDay"]= pd.Series(np.zeros(shape=(df.shape[0],)))
        df["ARestDay"]= pd.Series(np.zeros(shape=(df.shape[0],)))
        df["HAccS"]= pd.Series(np.zeros(shape=(df.shape[0],)))
        df["AAccS"]= pd.Series(np.zeros(shape=(df.shape[0],)))
        df["HAccST"]= pd.Series(np.zeros(shape=(df.shape[0],)))
        df["AAccST"]= pd.Series(np.zeros(shape=(df.shape[0],)))
        
        df["Sufficient"] = pd.Series(np.ones(shape=(df.shape[0],)))
        
        hscore = df['FTHG'].values
        ascore = df['FTAG'].values
        hconcede = df ['FTAG'].values
        aconcede = df['FTHG'].values
        hwin = df['HWin'].values
        awin = df['AWin'].values
        hlose = df['HLose'].values
        alose = df['ALose'].values
        hdraw = df['HDraw'].values
        adraw = df['ADraw'].values
        hmoral = df['HMoral'].values
        amoral = df['AMoral'].values
        
        rankRatio = (df['HAccP'].values+1) / (df['AAccP'].values +1)
        
        
        ftr = df["FTR"].values
        for i in range(df.shape[0]):
            sys.stdout.write("\r progress {}".format(i))
            sys.stdout.flush()
            if ftr[i] == 'H':
                hwin[i] = 1
                hmoral[i] = 3 * 1/rankRatio[i]
                alose[i]= 1
                amoral[i] = -3 * 1/rankRatio[i]
            elif ftr[i] == 'D':
                hdraw[i] = 1
                hmoral[i] = 1 * 1/rankRatio[i]
                adraw[i] = 1
                amoral[i] = 1 * rankRatio[i]
            else :
                hlose[i] = 1
                hmoral[i] = -3*rankRatio[i]
                awin [i] = 1
                amoral[i] = 3*rankRatio[i]
        
        
        df["HWin"]=hwin
        df["AWin"]=awin
        df["HDraw"]=hdraw
        df["ADraw"]=adraw
        df["HLose"]=hlose
        df["ALose"]=alose
        df["HScore"]=hscore
        df["AScore"]=ascore
        df["HConcede"]=hconcede
        df["AConcede"]=aconcede
        df["HMoral"] = hmoral
        df["AMoral"] = amoral
        
        
        
        for teamName in self.teamsMap.keys():
            print(teamName)
            team  = df[(df['HomeTeam']==teamName) | (df['AwayTeam'] == teamName)] 
            hometeam = team['HomeTeam'].values
            hwin = team["HWin"].values
            awin = team["AWin"].values
            hlose= team["HLose"].values
            alose = team["ALose"].values
            hdraw = team["HDraw"].values
            adraw = team["ADraw"].values
            hscore = team["HScore"].values
            ascore = team["AScore"].values
            hconcede = team["HConcede"].values
            aconcede = team["AConcede"].values
            hmoral = team["HMoral"].values
            amoral = team["AMoral"].values
            hrestday = team["HRestDay"].values
            arestday = team["ARestDay"].values
            
            hs= team["HS"].values
            as_ = team["AS"].values
            hst = team["HST"].values
            ast = team["AST"].values
            haccs = team["HAccS"].values
            aaccs = team["AAccS"].values
            haccst =team["HAccST"].values
            aaccst = team["AAccST"].values
            
            matchDate =team['Date'].values
            sufficient = team['Sufficient'].values
            teamMatchesDate = self.teamsData[teamName].sort('1',ascending=False)['1'].values
            
            restday = 0
            winsum =0 
            losesum=0
            drawsum=0
            scoresum =0
            concedesum=0
            accs = 0
            accst = 0
            moralsum = 0
            #print(team[['HomeTeam','AwayTeam','HWin']])
            for  i in range(0,n):
                if i < team.shape[0]:
                    scoresum = scoresum + (hscore[i] if hometeam[i] == teamName else ascore[i])
                    winsum = winsum + (hwin[i] if hometeam[i] == teamName else awin[i])
                    losesum= losesum + (hlose[i] if hometeam[i] == teamName else alose[i])
                    drawsum= drawsum + (hdraw[i] if hometeam[i] == teamName else adraw[i])
                    concedesum = concedesum+ (hconcede[i] if hometeam[i] == teamName else aconcede[i])
                    moralsum= moralsum+ (hmoral[i] if hometeam[i] == teamName else amoral[i])
                    accs = accs + (hs[i] if hometeam[i] == teamName else as_[i])
                    accst = accst + (hst[i] if hometeam[i] == teamName else ast[i])
                else:
                    # + 0
                    pass
            dateIndx = 0
            for j in range(team.shape[0]):
                while True:
                    if dateIndx >= teamMatchesDate.shape[0]:
                        sufficient[j] = False
                        break
                    if teamMatchesDate[dateIndx] < matchDate[j] :
                        restday = (matchDate[j] - teamMatchesDate[dateIndx])/np.timedelta64(1,'D')
                        break
                    else:
                        dateIndx = dateIndx + 1
                
                if j+n < team.shape[0]:                     
                    scoresum = scoresum + (hscore[j+n] if hometeam[j+n] == teamName else ascore[j+n])
                    winsum = winsum + (hwin[j+n] if hometeam[j+n] == teamName else awin[j+n])
                    losesum= losesum + (hlose[j+n] if hometeam[j+n] == teamName else alose[j+n])
                    drawsum= drawsum + (hdraw[j+n] if hometeam[j+n] == teamName else adraw[j+n])
                    concedesum = concedesum+ (hconcede[j+n] if hometeam[j+n] == teamName else aconcede[j+n])
                    moralsum= moralsum+ (hmoral[j+n] if hometeam[j+n] == teamName else amoral[j+n])
                    accs = accs + (hs[j+n] if hometeam[j+n] == teamName else as_[j+n])
                    accst = accst + (hst[j+n] if hometeam[j+n] == teamName else ast[j+n])
                else:
                    sufficient[j] = False
                    
                
                scoresum = scoresum - (hscore[j] if hometeam[j] == teamName else ascore[j])
                winsum = winsum - (hwin[j] if hometeam[j] == teamName else awin[j])
                losesum= losesum - (hlose[j] if hometeam[j] == teamName else alose[j])
                drawsum= drawsum - (hdraw[j] if hometeam[j] == teamName else adraw[j])
                concedesum = concedesum - (hconcede[j] if hometeam[j] == teamName else aconcede[j])
                moralsum= moralsum - (hmoral[j] if hometeam[j] == teamName else amoral[j])
                accs = accs - (hs[j] if hometeam[j] == teamName else as_[j])
                accst = accst - (hst[j] if hometeam[j] == teamName else ast[j])
                    
                if hometeam[j] == teamName:
                    hscore[j] = scoresum
                    hwin[j] = winsum
                    hlose[j] = losesum
                    hdraw[j] = drawsum
                    hconcede[j] = concedesum
                    hmoral[j] = moralsum
                    hrestday[j] = restday
                    haccs[j]=accs
                    haccst[j]=accst
                else:
                    ascore[j] = scoresum
                    awin[j] = winsum
                    alose[j] = losesum
                    adraw[j] = drawsum
                    aconcede[j] = concedesum
                    amoral[j] = moralsum
                    arestday[j] = restday
                    aaccs[j] = accs
                    aaccst[j] = accst
            team["HWin"]=hwin
            team["AWin"]=awin
            team["HDraw"]=hdraw
            team["ADraw"]=adraw
            team["HLose"]=hlose
            team["ALose"]=alose
            team["HScore"]=hscore
            team["AScore"]=ascore
            team["HConcede"]=hconcede
            team["AConcede"]=aconcede
            team["HMoral"] = hmoral
            team["AMoral"] = amoral
            team['Sufficient'] = sufficient
            team['HAccS'] = haccs
            team['AAccS'] = aaccs
            team['HAccST'] = haccst
            team['AAccST'] = aaccst
        
            #print(team[['HomeTeam','AwayTeam','HWin']])
            df.update(team)
            #print(df[['HomeTeam','AwayTeam','HAccP','AAccP']])
        self.df =df
        return df
    def initTeamData(self):
        self.teamsData={}
        self.teamsById={}
        for name in self.teamsMap.keys():
            self.readTeamMatch(name)
    def preprocessData(self,recent=5):
        self.initTeamData()
        self.initRanking()
        self.initRecentData(recent)
            

/home/y/scikit-learn/sklearn/cross_validation.py:43: DeprecationWarning: This module has been deprecated in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import SGD, Adadelta, Adagrad
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping
def createModel(hidSize, inputDim):
    model = Sequential()
    model.add(Dense(hidSize[0], input_dim=inputDim, init='glorot_normal'))
    model.add(Activation('sigmoid'))
    model.add(Dropout(0.5))

    model.add(Dense(hidSize[1], init='glorot_normal'))
    model.add(Activation('sigmoid'))
    model.add(Dropout(0.5))

    model.add(Dense(3, init='glorot_normal'))
    model.add(Activation('softmax'))
    sgd = SGD(lr=1, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer='adadelta')
    return model
earlyCallback = EarlyStopping(patience=20,verbose=1)

Using Theano backend.
/home/y/anaconda3/lib/python3.5/site-packages/theano/tensor/signal/downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")


In [20]:
import urllib 
import math
from datetime import date
import re
class DataGrapper:
    def updateTeams(self,from_year,to_year):
        df = pd.read_csv('teamId.csv',index_col=0)
        teamMatchUrl = 'http://app.gooooal.com/teamAllMatch.do?sid={}&tid={}&t=0&lang=tr'
        teams = df.values
        for i in range(teams.shape[0]):
            thisDf = None
            for year in range(from_year,to_year):
                url = teamMatchUrl.format(year,teams[i,0])
                print(url)
                tdf = pd.read_html(url,attrs={'id':'fb_match_odds_result_table_0_0'},encoding='UTF-8')[0]
                tdf = tdf.drop(df.index[[0]])
                if thisDf is None:
                    thisDf=tdf
                else:
                    thisDf =thisDf.append(tdf)
            print(teams[i,1])
            thisDf.to_csv("teams/"+teams[i,1]+".csv",index=False)

    def getOdds(self,date):
        dateStr = "{}-{}-{}".format(date.year,date.month,date.day)
        print("get {}".format(dateStr))
        url = 'http://info.livescore123.com/1x2/companyhistory.aspx?id=432&company=HK+Jockey+Club'
        data = {'matchdate': dateStr,'Submit': 'Search','key': ""}
        f = urllib.request.urlopen(
                url     = url,
                data    = urllib.parse.urlencode(data).encode('utf8')
            )
        raw= f.read()
        df = pd.read_html(raw,attrs={'class':'schedule'},header=0)[0]
        df = df[df['League'] == 'ENG PR'] 
        df = df [['Home','Away','HW','D','AW']]
        return df 
    def updateOdds(self,greater_than,origin, prev_odds, out_odds):
        footballdata = pd.read_csv(origin)
        footballdata['Date'] = pd.to_datetime(footballdata['Date'],dayfirst=True)
        if 'JocH' not in footballdata:
            footballdata['JocH']=pd.Series(np.zeros(shape=(footballdata.shape[0],)), index=footballdata.index) 
            footballdata['JocD'] = pd.Series(np.zeros(shape=(footballdata.shape[0],)), index=footballdata.index) 
            footballdata['JocA'] = pd.Series(np.zeros(shape=(footballdata.shape[0],)), index=footballdata.index)         
        dates = footballdata['Date'].drop_duplicates()
        prev_odds = pd.read_csv(prev_odds)
        prev_odds['Date'] = pd.to_datetime(prev_odds['Date'],dayfirst=True)
        footballdata = footballdata.merge(prev_odds,left_on=['Date','HomeTeam','AwayTeam'],
                                 right_on=['Date','HomeTeam','AwayTeam'],how='left')
        dates  = dates[dates > greater_than]
        for d in dates:
            odds = None
            err_count =0
            while err_count < 5:
                try:
                    odds = self.getOdds(d)
                    break
                except Exception as e:
                    print (e)
                err_count = err_count +1
            tempdf = footballdata[footballdata['Date'] == d]
            tempdf = tempdf.sort(columns='HomeTeam')
            odds = odds.sort(columns='Home')
            tempdf['JocH']= odds['HW'].values
            tempdf['JocD'] = odds['D'].values
            tempdf['JocA'] = odds['AW'].values
           # print(odds)
            footballdata.update(tempdf)
        footballdata.to_csv(out_odds,index)
    def readLastestMatches(self):
        url = 'http://bet.hkjc.com/football/odds/odds_had.aspx?ci=en-US'
        content= urllib.request.urlopen(url).read().decode('utf-8')
        #print(content)
        p2 = '<img src="/nas/jcbwinfo/football/info/images/.*?" alt="(.*?)"'
        p1= '<img src="/nas/jcbwinfo/football/info/images/.*?" alt="(.*?)".*?>'
        league = []
        for m in re.finditer(p1,content,flags=re.M):
            league.append(m.group(1))
        df= pd.read_html(url,attrs={'class':'tOdds'},header =0)[1]
        df = df.dropna(subset=['Expected StopSelling Time'])
        df['Unnamed: 1']=league
        ta = df['Teams(Home vs Away)'].values
        print(ta.shape)
        homeAways =[]
        for v in ta:
            homeAways.append([e.strip() for e in (v.split(' vs '))])
        nha = np.array(homeAways)
        df['HomeTeam']=nha[:,0]
        df['AwayTeam']=nha[:,1]
        rawTimes = df['Expected StopSelling Time'].values
        timesStr = [(v.split()[0])+'/2016' for v in rawTimes]
        print(timesStr)
        df['Date'] = pd.to_datetime(timesStr,dayfirst=True)
        df= df[df['Unnamed: 1']=='Eng Premier']
        df["JocH"] =df["Odds"].values
        df["JocD"]= df["Unnamed: 7"].values
        df["JocA"] = df["Unnamed: 8"].values
        df = df[["HomeTeam","AwayTeam","Date","JocH","JocD","JocA"]]
        return df.dropna(subset=['JocD'])
        #TODO: find epl name, -> epl only -> sort name -> return h,a,date,odds

In [21]:
tg = DataGrapper()
df =tg.readLastestMatches()

(60,)
['-/2016', '-/2016', '-/2016', '02/04/2016', '02/04/2016', '02/04/2016', '02/04/2016', '02/04/2016', '02/04/2016', '02/04/2016', '02/04/2016', '02/04/2016', '02/04/2016', '02/04/2016', '02/04/2016', '02/04/2016', '02/04/2016', '02/04/2016', '02/04/2016', '02/04/2016', '02/04/2016', '02/04/2016', '02/04/2016', '02/04/2016', '02/04/2016', '02/04/2016', '02/04/2016', '02/04/2016', '02/04/2016', '02/04/2016', '02/04/2016', '02/04/2016', '02/04/2016', '02/04/2016', '02/04/2016', '02/04/2016', '02/04/2016', '02/04/2016', '02/04/2016', '02/04/2016', '02/04/2016', '02/04/2016', '02/04/2016', '02/04/2016', '02/04/2016', '02/04/2016', '03/04/2016', '03/04/2016', '03/04/2016', '03/04/2016', '03/04/2016', '03/04/2016', '03/04/2016', '03/04/2016', '03/04/2016', '03/04/2016', '03/04/2016', '03/04/2016', '03/04/2016', '03/04/2016']


In [23]:
df.to_csv("dataSet/future.csv",index=False)

In [24]:
from bs4 import BeautifulSoup
import datetime
import re
def getContentSoup (url):
        count = 0
        content=""
        while(count < 10):
            try:
                content = urllib.request.urlopen(url).read().decode('utf-8')
                break
            except:
                print("re-try")
                time.sleep(5)
                count = count+1
            if count ==10:
                raise Exception("error {}".format(url))
        soup = BeautifulSoup(content,'html.parser')
        return soup
def grapMatchInfo(url):

    from bs4 import BeautifulSoup
    import urllib 
    import math
    from datetime import date
    import datetime
    import re
    import time
    def getContentSoup (url):
        count = 0
        content=""
        while(count < 10):
            try:
                content = urllib.request.urlopen(url).read().decode('utf-8')
                break
            except:
                print("re-try")
                sys.stdout.flush()
                time.sleep(5)
                count = count+1
            if count ==10:
                raise Exception("error {}".format(url))
        soup = BeautifulSoup(content,'html.parser')
        return soup
    
    def summary():
        soup = getContentSoup(url)
        temp = soup.find("div","teams").text.split("\n")[1].split(',')[2:]
        tempstr = " ".join("".join(temp).strip().split(" ")[0:3])
        date = datetime.datetime.strptime(tempstr,"%B %d %Y")
        
        poss = [float(e.text[0:-1]) for e in soup.select("div#summary_possessions text")]
        #extract home id
        homeurl = soup.select("a.tabs-primary__tab-link")[1].get('href')
        homeId = -1 
        for m in re.finditer('/statszone/.*?/matches/.*?/team-stats/(\d+?)/OVERALL_01',homeurl,re.M):
            homeId = m.group(1)
        return date , poss,homeId
    def getTeamStat(homeId):
        #passing third
        atkthirdUrl = '/team-stats/{}/1_PASS_08'
        soup = getContentSoup(url+atkthirdUrl.format(homeId))
        awayurl=(soup.select("ul#statzone_pitch_team_filter li.last a")[0].get('href'))
        awayId = -1
        for m in re.finditer('/statszone/.*?/matches/.*?/team-stats/(\d+?)/1_PASS_08',awayurl,re.M):
            awayId = m.group(1)
        #name
        awayName = soup.select("ul#statzone_pitch_team_filter li.last a")[0].text
        homeName = soup.select("ul#statzone_pitch_team_filter li.first a")[0].text
        #attacking third
        h_atkThird = [ int(e.text) for e in soup.select("div.title-wrapper span")]
        soup = getContentSoup(url+atkthirdUrl.format(awayId))
        a_atkThird = [ int(e.text) for e in soup.select("div.title-wrapper span")]
        #interceptions
        interUrl = '/team-stats/{}/3_DEFENCE_02'
        soup= getContentSoup(url+interUrl.format(homeId))
        h_interception = len(soup.select("svg#pitch image.pitch-object"))
        soup = getContentSoup(url+interUrl.format(awayId))
        a_interception = len(soup.select("svg#pitch image.pitch-object"))
        #redArea
        #redUrl = '/team-stats/{}/0_SHOT_055#tabs-wrapper-anchor#:V4wYu_PQ53MINA'
        #soup= getContentSoup('http://www.fourfourtwo.com/statszone/8-2014/matches/755673/team-stats/3/0_SHOT_055#tabs-wrapper-anchor#:j9_8SoNqObMINA')
        #print(soup.prettify())
        #print(soup.find_all("line"))
        return homeName, awayName, h_atkThird, a_atkThird, h_interception,a_interception
        
    date,poss,homeId = summary()
    homeName, awayName, h_atkThird, a_atkThird, h_interception,a_interception=  getTeamStat(homeId)
    return  date ,homeName, awayName ,poss, h_atkThird, a_atkThird, h_interception,a_interception
    
    


In [47]:
import os, http.cookiejar, urllib.request
from cookielib import Cookie, CookieJar

def getReferee():
    cj = CookieJar()
    # Cookie(version, name, value, port, port_specified, domain, 
    # domain_specified, domain_initial_dot, path, path_specified, 
    # secure, discard, comment, comment_url, rest)
    c = Cookie(None, 'asdf', None, '80', '80', 'www.foo.bar', 
           None, None, '/', None, False, False, 'TestCookie', None, None, None)
    cj.set_cookie(c)
    print (cj)
    url = 'http://www.premierleague.com/content/premierleague/en-gb/referees/appointments.html?paramSearchContext=MATCH_WEEK&paramMatchWeek=31'
    return
    req = urllib.request.Request(url)
    req.add_header('Accept-Language','en-US;q=0.6,en;q=0.4')
    content = urllib.request.urlopen(req).read().decode('utf-8')
    print(content)
    #soup= getContentSoup(url)
    #print(soup.prettify())
    dfs = pd.read_html(url)
    dfs[0]["Date"] =datetime.date(2016,4,2)
    dfs[1]["Date"] = datetime.date(2016,4,3)
    print(dfs)



ImportError: No module named 'cookielib'

In [48]:
getReferee()










<!--[if lt IE 7]>      <html lang="zh" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html lang="zh" class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html lang="zh" class="no-js lt-ie9"> <![endif]-->
<!--[if IE 9]>         <html lang="zh" class="no-js lt-ie10"> <![endif]-->
<!--[if gt IE 8]><!--> <html lang="zh"> <!--<![endif]-->

    
 

<head>
    <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">
    <meta http-equiv="Content-Type" content="text/html; charset=utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1">
    <meta name="apple-mobile-web-app-capable" content="yes">
    <meta name="apple-mobile-web-app-status-bar-style" content="black-translucent">
    <meta name="HandheldFriendly" content="true"> 
    <meta name="MobileOptimized" content="320">
    <meta name="description" content="">
    <meta name="author" content="">
    




 


    <meta property="wb:webmaster" con

ValueError: No tables found

In [6]:
from IPython.parallel import Client
import time
from IPython.display import clear_output
import sys
def getYearMatch(year):
    
    listurl ='http://www.fourfourtwo.com/statszone/results/8-{}'.format(year)
    baseurl = 'http://www.fourfourtwo.com'
    soup = getContentSoup(listurl)
    tot_links = [e.get('href') for e in  (soup.select("td.link-to-match a"))]
    dates = []
    h_names = []
    a_names =[]
    posses = []
    h_atks = []
    a_atks=[]
    h_ins=[]
    a_ins=[]
    def workerFunc(args):
        import sys
        links =args[0]
        grapMatchInfo = args[1]
        print("start working")
        dates = []
        h_names = []
        a_names =[]
        posses = []
        h_atks = []
        a_atks=[]
        h_ins=[]
        a_ins=[]
        for indx, link in enumerate(links):
            print("task: {}/{}".format(indx,len(links)))
            sys.stdout.flush()
            try:
                date ,homeName, awayName ,poss, h_atkThird, a_atkThird, h_interception,a_interception = grapMatchInfo(baseurl + link)
                print("{} {} {}".format(date,homeName,awayName))
                dates.append(date)
                h_names.append(homeName)
                a_names.append(awayName)
                posses.append(poss)
                h_atks.append(h_atkThird)
                a_atks.append(a_atkThird)
                h_ins.append(h_interception)
                a_ins.append(a_interception)
            except Exception as e:
                print(e)
                print("error")
                break
            
        print("finish")
        return [dates,h_names,a_names,posses,h_atks,a_atks,h_ins,a_ins]
    def finishCallback(returnArgs):
        print("complete")
        dates = []
        h_names = []
        a_names =[]
        posses = []
        h_atks = []
        a_atks=[]
        h_ins=[]
        a_ins=[]
        for args in returnArgs:
            dates=dates+args[0]
            h_names = h_names + args[1]
            a_names = a_names + args[2]
            posses = posses+args[3]
            h_atks=h_atks + args[4]
            a_atks = a_atks + args[5]
            h_ins = h_ins + args[6]
            a_ins = a_ins + args[7]
        return dates,h_names,a_names,posses,h_atks,a_atks,h_ins,a_ins
        
    n_work = 4
    pool = Client()
    print(pool.ids)
    each_n = int(len(tot_links)/n_work)
    works = [tot_links[i*each_n:(i+1)*each_n] for i in range(n_work)]
    works[0].append(tot_links[4*each_n:])
    results = []
    for i in range(n_work):
        results.append(pool[i].apply_async(f=workerFunc,args=[works[i],grapMatchInfo,getContentSoup]))
    def wait_watching_stdout(ars, dt=5, truncate=500):
        while True:
            allReady = True 
            printed = False
            for ar in ars:
                if not ar.ready():
                    allReady = False
                    stdouts = ar.metadata['stdout']
                    if not any(stdouts):
                        continue
                    if printed == False:
                        clear_output()
                        printed = True
                    # clear_output doesn't do much in terminal environments
                    print ('-' * 30)
                    print ("%.3fs elapsed" % ar.elapsed)
                    print ("")
                    print(ar._targets)
                    #print ("[ stdout %2i ]" % (ar._targets))
                    print(ar.metadata['stdout'][-truncate:])
                    sys.stdout.flush()
            if allReady == True:
                return
            time.sleep(dt)      
    wait_watching_stdout(results)
    returnArgs = [ e.get() for e in results]
    pool.close()
    return finishCallback(returnArgs)

    
    

/home/y/anaconda3/lib/python3.5/site-packages/IPython/parallel.py:13: ShimWarning: The `IPython.parallel` package has been deprecated. You should import from ipyparallel instead.
  "You should import from ipyparallel instead.", ShimWarning)


In [7]:
from IPython.parallel import Client
import time
from IPython.display import clear_output
import sys
def getYearMatch(year):
    
    listurl ='http://www.fourfourtwo.com/statszone/results/8-{}'.format(year)
    baseurl = 'http://www.fourfourtwo.com'
    soup = getContentSoup(listurl)
    tot_links = [e.get('href') for e in  (soup.select("td.link-to-match a"))]
    dates = []
    h_names = []
    a_names =[]
    posses = []
    h_atks = []
    a_atks=[]
    h_ins=[]
    a_ins=[]
    print("start working")
    for indx, link in enumerate(tot_links):
        clear_output()
        print("task: {}/{}".format(indx,len(tot_links)))
        sys.stdout.flush()
        try:
            date ,homeName, awayName ,poss, h_atkThird, a_atkThird, h_interception,a_interception = grapMatchInfo(baseurl + link)
            print("{} {} {}".format(date,homeName,awayName))
            dates.append(date)
            h_names.append(homeName)
            a_names.append(awayName)
            posses.append(poss)
            h_atks.append(h_atkThird)
            a_atks.append(a_atkThird)
            h_ins.append(h_interception)
            a_ins.append(a_interception)
        except Exception as e:
            print(e)
            print("error")
            break
            
        print("finish")
    
 
    return dates,h_names,a_names,posses,h_atks,a_atks,h_ins,a_ins

In [ ]:
dates,h_names,a_names,posses,h_atks,a_atks,h_ins,a_ins= getYearMatch(2012)

task: 4/380


In [160]:
def getTempMatch():
    
    tot_links = ['/statszone/8-2015/matches/803167',
                 '/statszone/8-2015/matches/803170',
                 '/statszone/8-2015/matches/803168']
                 
    baseurl = 'http://www.fourfourtwo.com'
    dates = []
    h_names = []
    a_names =[]
    posses = []
    h_atks = []
    a_atks=[]
    h_ins=[]
    a_ins=[]
    print("start working")
    for indx, link in enumerate(tot_links):
        clear_output()
        print("task: {}/{}".format(indx,len(tot_links)))
        sys.stdout.flush()
        try:
            date ,homeName, awayName ,poss, h_atkThird, a_atkThird, h_interception,a_interception = grapMatchInfo(baseurl + link)
            print("{} {} {}".format(date,homeName,awayName))
            dates.append(date)
            h_names.append(homeName)
            a_names.append(awayName)
            posses.append(poss)
            h_atks.append(h_atkThird)
            a_atks.append(a_atkThird)
            h_ins.append(h_interception)
            a_ins.append(a_interception)
        except Exception as e:
            print(e)
            print("error")
            break
            
        print("finish")
    
 
    return dates,h_names,a_names,posses,h_atks,a_atks,h_ins,a_ins

In [162]:
n_poss =np.array(posses).T
h_poss = n_poss[0,:]
a_poss = n_poss[1,:]
n_h_atks = np.array(h_atks).T
n_a_atks = np.array(a_atks).T
h_atks_ok = n_h_atks[0,:]
h_atks_tot = n_h_atks[1,:]
a_atks_ok = n_a_atks[0,:]
a_atks_tot = n_a_atks[1,:]
res = np.array([dates,h_names,a_names,h_poss,a_poss,h_atks_ok,h_atks_tot,
          a_atks_ok,a_atks_tot, h_ins,a_ins]).ravel().reshape(11,len(dates)).T
df = pd.DataFrame(res, columns=['Date','HomeTeam','AwayTeam','H_Poss','A_Poss','H_tkPass_OK','H_atkPass_tot',
                          'A_atkPass_Ok','A_atkPass_tot','H_ins','A_ins'])

In [174]:
df.sort(columns="Date",ascending=False)

/home/y/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':
/home/y/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:6: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,Date,HomeTeam,AwayTeam,H_Poss,A_Poss,H_tkPass_OK,H_atkPass_tot,A_atkPass_Ok,A_atkPass_tot,H_ins,A_ins
2,2015-08-08 00:00:00,Chelsea,Swansea City,52.5,47.5,121,174,75,110,13,12
297,2015-08-08 00:00:00,Manchester United,Tottenham Hotspur,50.1,49.9,106,143,52,98,19,29
298,2015-08-08 00:00:00,Bournemouth,Aston Villa,59.3,40.7,134,184,73,102,16,28
1,2015-08-08 00:00:00,Everton,Watford,66.7,33.3,128,195,71,117,11,37
0,2015-08-08 00:00:00,Leicester City,Sunderland,44.3,55.7,67,112,57,91,18,11
299,2015-08-08 00:00:00,Norwich City,Crystal Palace,62.9,37.1,97,141,81,113,7,12
295,2015-08-09 00:00:00,Arsenal,West Ham United,62.1,37.9,179,213,46,96,18,12
294,2015-08-09 00:00:00,Newcastle United,Southampton,55.9,44.1,92,134,72,115,15,34
296,2015-08-09 00:00:00,Stoke City,Liverpool,46.8,53.2,51,96,82,132,13,13
293,2015-08-10 00:00:00,West Bromwich Albion,Manchester City,31.1,68.9,71,98,212,247,16,14


In [175]:
df2.to_csv('dataSet/match2015.csv',index=False)